# 12.24 Yield management 1
Williams, H. Paul. Model building in mathematical programming. John Wiley & Sons, 2013.

3 つの期間に関して最適化を行う。各期間では、最大 6 機まで航空機を使用でき、1 フライトごとに 50000 ポンドかかる。

各航空機には、デフォルトで以下の座席がある。

- 37 first class
- 38 business class
- 47 economy class

最大 10% までの座席は隣接クラスに変更可能。

3 つの価格レベルを用意するので、各機関においてどのレベルを使用するのか決定する。

## モデリング
### 問題設定について分からなかった点
- 価格レベルは同じ期間であれば全てのクラスで共通？
- 需要はすべて満たさなければいけない？（meet commitments の意味）

### モデル化の仕方で分からなかった点
- 隣接クラスへの変更をどう扱うか？

まずは簡単のために以下の設定で最適化を行う。

- シナリオ 1 のみ
- 価格レベルは全クラス共通
- 需要はすべて満たす
- 隣接クラスへの変更は行わない

In [1]:
import numpy as np
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

## 集合の読み込み

In [2]:
TIME_IDX = [1, 2, 3]
CLASSES = ["First", "Business", "Economy"]
SCENARIOS = [1, 2, 3]
OPTIONS = [1, 2, 3]

## パラメータの読み込み

In [3]:
n_planes = 6
flight_uc = 50000

In [4]:
n_seats_default = {
    "First": 37,
    "Business": 38,
    "Economy": 47,
}

In [5]:
prices = pd.DataFrame.from_dict({
    (1, 'First'): [1200, 1400, 1500],
    (1, 'Business'): [900, 1100, 820],
    (1, 'Economy'): [500, 700, 480],
    (2, 'First'): [1000, 1300, 900],
    (2, 'Business'): [800, 900, 800],
    (2, 'Economy'): [300, 400, 470],
    (3, 'First'): [950, 1150, 850],
    (3, 'Business'): [600, 750, 500],
    (3, 'Economy'): [200, 350, 450],
}).T
prices.index.names = ["Option", "Class"]
prices.columns = [1, 2, 3]
prices

1     2     3
Option Class                     
1      First     1200  1400  1500
       Business   900  1100   820
       Economy    500   700   480
2      First     1000  1300   900
       Business   800   900   800
       Economy    300   400   470
3      First      950  1150   850
       Business   600   750   500
       Economy    200   350   450

In [6]:
demands = pd.DataFrame.from_dict({
    (1, 'First'): [10, 20, 30],
    (1, 'Business'): [20, 42, 40],
    (1, 'Economy'): [45, 50, 50],
    (2, 'First'): [15, 25, 35],
    (2, 'Business'): [25, 45, 50],
    (2, 'Economy'): [55, 52, 60],
    (3, 'First'): [20, 35, 40],
    (3, 'Business'): [35, 46, 55],
    (3, 'Economy'): [60, 60, 80],
}).T
demands.index.names = ["Option", "Class"]
demands.columns = [1, 2, 3]
demands

1   2   3
Option Class               
1      First     10  20  30
       Business  20  42  40
       Economy   45  50  50
2      First     15  25  35
       Business  25  45  50
       Economy   55  52  60
3      First     20  35  40
       Business  35  46  55
       Economy   60  60  80

### 変数

In [7]:
is_selected = LpVariable.dicts("price_option", (TIME_IDX, OPTIONS), lowBound=0, upBound=1, cat="Integer")
n_flights = LpVariable.dicts("n_flights", (TIME_IDX), lowBound=0, upBound=n_planes, cat="Integer")

### 目的関数

In [8]:
flight_cost = flight_uc * lpSum(n_flights)
total_sales = lpSum(demands.loc[(option, c), t] * prices.loc[(option, c), t] * is_selected[t][option]
                    for c in CLASSES for t in TIME_IDX for option in OPTIONS)
total_profit = total_sales - flight_cost
total_profit

-50000*n_flights_1 + -50000*n_flights_2 + -50000*n_flights_3 + 52500*price_option_1_1 + 51500*price_option_1_2 + 52000*price_option_1_3 + 109200*price_option_2_1 + 93800*price_option_2_2 + 95750*price_option_2_3 + 101800*price_option_3_1 + 99700*price_option_3_2 + 97500*price_option_3_3 + 0

In [9]:
model = LpProblem("Yield_management", LpMaximize)
model.setObjective(total_profit)

### 制約条件

In [10]:
# Optioon のうちどれか一つを選ぶ
for t in TIME_IDX:
    model += lpSum(is_selected[t]) == 1

In [11]:
# フライト数に応じた席数が需要を満たす
for t in TIME_IDX:
    for c in CLASSES:
        for option in OPTIONS:
            model += demands.loc[(option, c), t] * is_selected[t][option] <= n_flights[t] * n_seats_default[c]

In [12]:
model.solve()

1

In [13]:
model.objective.value()

13500.0

In [14]:
# フライト数
pd.Series({t: n_flights[t].value() for t in TIME_IDX})

1    1.0
2    2.0
3    2.0
dtype: float64

In [16]:
# 価格オプション
# 行：期間
# 列：オプション
pd.DataFrame({option: {t: is_selected[t][option].value() for t in TIME_IDX} for option in OPTIONS})

,1,2,3
1,1.0,0.0,0.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
